<a href="https://colab.research.google.com/github/Farida-AI-FutureProof/5m-data-2.3-data-encoding-creation-flow/blob/main/Assignment_2_3_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "protobuf<6.0.0" grpcio grpcio-tools thriftpy2 --force-reinstall --upgrade


INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: ply
    Found existing installation: ply 3.11
    Uninstalling ply-3.11:
      Successfully uninstalled ply-3.11
  Attempting un

In [2]:
import google
import grpc
import thriftpy2
import google.protobuf

print("grpc version:", grpc.__version__)
print("protobuf version:", google.protobuf.__version__)


grpc version: 1.76.0
protobuf version: 5.29.5


In [3]:
!pip install thriftpy2 grpcio grpcio-tools


In [4]:
import os

directory = "schema"
if not os.path.exists(directory):
    os.makedirs(directory)

file_content = """struct Student {
  1: required string name,
  2: required i64 age,
  3: required list<string> courses
}

service School {
  Student enrollCourse(1: Student student, 2: string courseName)
}"""

with open(os.path.join(directory, "student.thrift"), "w") as f:
    f.write(file_content)

print(f"Writing {os.path.join(directory, 'student.thrift')}")

Writing schema/student.thrift


In [5]:
%%writefile student_server.py
import thriftpy2
from thriftpy2.rpc import make_server

# Load the Thrift schema
student_thrift = thriftpy2.load("schema/student.thrift", module_name="student_thrift")

class SchoolService:
    def enrollCourse(self, student, courseName):
        student.courses.append(courseName)
        return student

if __name__ == "__main__":
    print("Starting Thrift server on port 6000...")
    server = make_server(student_thrift.School, SchoolService(), "0.0.0.0", 6000)
    server.serve()


Writing student_server.py


In [6]:
!python student_server.py


Starting Thrift server on port 6000...
Traceback (most recent call last):
  File "/content/student_server.py", line 15, in <module>
    server.serve()
  File "/usr/local/lib/python3.12/dist-packages/thriftpy2/server.py", line 76, in serve
    self.trans.listen()
  File "/usr/local/lib/python3.12/dist-packages/thriftpy2/transport/socket.py", line 221, in listen
    self.sock.bind(addr)
OSError: [Errno 98] Address already in use


In [7]:
%%writefile student_client.py
import thriftpy2
from thriftpy2.rpc import make_client

# Load schema
student_thrift = thriftpy2.load("schema/student.thrift", module_name="student_thrift")

# Create client
client = make_client(student_thrift.School, "localhost", 6000)

# Create student object
student = student_thrift.Student(
    name="Farida",
    age=48,
    courses=["Data Science"]
)

# Call service
updated_student = client.enrollCourse(student, "AI Engineering")

print("Updated student:")
print(updated_student)


Writing student_client.py


In [8]:
!python student_client.py


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/thriftpy2/transport/socket.py", line 98, in open
    self.sock.connect(addr)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/student_client.py", line 8, in <module>
    client = make_client(student_thrift.School, "localhost", 6000)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/thriftpy2/rpc.py", line 54, in make_client
    transport.open()
  File "thriftpy2/transport/buffered/cybuffered.pyx", line 36, in thriftpy2.transport.buffered.cybuffered.TCyBufferedTransport.open
  File "/usr/local/lib/python3.12/dist-packages/thriftpy2/transport/socket.py", line 105, in open
    raise TTransportException(
thriftpy2.transport.base.TTransportException: TTransportException(type=1, message="Could not connect to ('localhost'

In [9]:
%%writefile schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  string author = 2;
  int32 page_count = 3;
}

service Library {
  rpc checkoutBook(Book) returns (Book);
}


Writing schema/book.proto


In [10]:
!python -m grpc_tools.protoc -I=schema --python_out=. --grpc_python_out=. schema/book.proto


In [11]:
%%writefile book_server.py
import grpc
from concurrent import futures
import book_pb2
import book_pb2_grpc

class LibraryService(book_pb2_grpc.LibraryServicer):
    def checkoutBook(self, request, context):
        # Simply return the same Book message
        return book_pb2.Book(
            title=request.title,
            author=request.author,
            page_count=request.page_count
        )

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=5))
    book_pb2_grpc.add_LibraryServicer_to_server(LibraryService(), server)
    server.add_insecure_port("[::]:50051")
    server.start()
    print("gRPC server running on port 50051...")
    server.wait_for_termination()

if __name__ == "__main__":
    serve()


Writing book_server.py


In [12]:
!python book_server.py


gRPC server running on port 50051...
object address  : 0x7a732dbef460
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [13]:
%%writefile book_client.py
import grpc
import book_pb2
import book_pb2_grpc

# Connect to the gRPC server
channel = grpc.insecure_channel("localhost:50051")
stub = book_pb2_grpc.LibraryStub(channel)

# Create Book message
book = book_pb2.Book(
    title="Data Engineering 101",
    author="Farida",
    page_count=250
)

# Call RPC
returned_book = stub.checkoutBook(book)

print("Returned Book:")
print(returned_book)


Writing book_client.py


In [14]:
!python book_client.py


^C
